In [ ]:
from urllib import *
import json
import regex as re
import PySimpleGUI as sg
import webbrowser
import sys
import pprint
from dataclasses import dataclass
import hyperlink
import requests
import time

In [ ]:
def makeSelectRequest(say, start):
    wt = "wt=json"
    connection = requests.get('http://localhost:8983/solr/comment/select', params={"q": say, 'wt' : 'json', 'rows' : 10, 'start' : start, 'indent' : True})
    response=connection.json()
    num = response['response']['numFound']
    print("Query time: "+str(response['responseHeader']['QTime'])+" ms")
    return (num, response)

In [ ]:
def makeSpellRequest(say, start):
    wt = "wt=json"
    connection = requests.get('http://localhost:8983/solr/comment/spell', params={"q": say, 'wt' : 'json', 'rows' : 10, 'start' : start, 'indent' : True})
    response=connection.json()    
    print("Query time: "+str(response['responseHeader']['QTime'])+" ms")
    if (response['spellcheck']['correctlySpelled']==True):
        print("The word spelling is correct")
        return
    if not (response['spellcheck']['suggestions']):
        print("The word correction does not exist in our database")
        return
    for i in response['spellcheck']['suggestions'][1]['suggestion']:
        print(i['word'], " ", i['freq'])
        return

In [ ]:
def displaySearch(num, response):
    lay = []
    records = 0
    page_num = int((num+10-1)/10)

    for i in range(len(response['response']['docs'])):
        lay.append(response['response']['docs'][i])

    for yiz in lay:
        records+=1
        print("Restaurant: "+yiz['restaurant'])
        if(len(yiz['type'])>1):
            print("Type: "+yiz['type'][0]+", "+(yiz['type'][1].replace(" ", "")))
        else:
            print("Type: "+yiz['type'][0])
        print("Reviewer: "+yiz['reviewer'][0])
        print("Rating: "+str(yiz['rating']))
        print("Comments: "+yiz['comment'])
        print("Pred_label: "+yiz['pred_label'])
        print("Score: "+str(yiz['score']))
        print("------------------------------------------------------------------------------------------------------")
    return records

In [ ]:
def pageCount(num):
    if(num%10==0):
        total = num//10
    else:
        total = num//10+1
    return total

In [ ]:
def displayPage(current, total, entries, num):
    print("Page: "+str(current//10+1), "Total pages: "+str(total))
    print("Number of entries in this page: "+str(entries)+" of "+str(num))
    return

In [ ]:
def firstPageCheck(start):
    if((start-10)<0):
        print("You are at first page!!!")
        start=0
        return True
    
def lastPageCheck(start, num): 
    if((start+10)>=num):
        print("You are at last page!!!")
        return -1
            
    if(num<11):
        start = 0
    elif(num>10):
        if(num%10==0):
            start = num//10*10-10
        else:
            start = num//10*10
            
    return start

In [ ]:
layout = [[sg.Text('Search bar'), sg.InputText(size=(30,10)), sg.Button('Search', bind_return_key=True, key="_SEARCH_")],
          [sg.Combo(['Search', 'Spell check'], default_value='Search', key="_type_")],
          [sg.Output(font = ["Arial", 16], size=(180,41), key = '_output_')],
          [sg.Button('Previous page', bind_return_key=True, key="_prev_"), 
           sg.Button('Next page', bind_return_key=True, key="_next_"),
           sg.Button("Go to first page", bind_return_key=True, key="_first_"),
           sg.Button('Go to last page', bind_return_key=True, key='_last_')]]

window = sg.Window("Demo Search Engine", layout, size=(1440, 850))

while True:
    event, values = window.read()
    
####################################################################
    if event == sg.WIN_CLOSED:
        break
        
####################################################################        
    if event == '_SEARCH_':
        
        window['_output_'].update(value='')
        
        lay = []
        
        say = str(values[0])

        start = 0
            
        if(values['_type_']=='Spell check'):
            makeSpellRequest(say, start)
        
        else:
            num, response = makeSelectRequest(say, start)

            entries = displaySearch(num, response)
            
            total = pageCount(num)
            
            displayPage(start, total, entries, num)

####################################################################
    if event == '_next_':
        
        start+=10
        
        if(lastPageCheck(start, num)==-1):
            start-=10
            continue    
        
        window['_output_'].update(value='')

        num, response = makeSelectRequest(say, start)
        
        entries = displaySearch(num, response)

        total = pageCount(num)
        
        displayPage(start, total, entries, num)
        
####################################################################         
    if event == '_prev_':
        
        if(firstPageCheck(start)==True):
            continue
            
        start-=10
        
        window['_output_'].update(value='')
      
        num, response = makeSelectRequest(say, start)
        
        entries = displaySearch(num, response)
            
        total = pageCount(num)
                
        displayPage(start, total, entries, num)
        
####################################################################
    if event == '_first_':
        
        if(firstPageCheck(start)==True):
            continue
            
        start = 0
        
        window['_output_'].update(value='')
        
        num, response = makeSelectRequest(say, start)
        
        entries = displaySearch(num, response)
            
        total = pageCount(num)
        
        displayPage(start, total, entries, num)
                
####################################################################
    if event == '_last_':
     
        if(lastPageCheck(start, num)==-1):
            continue
                  
        start = lastPageCheck(start, num)
        
        window['_output_'].update(value='')
        
        num, response = makeSelectRequest(say, start)
       
        entries = displaySearch(num, response)
            
        total = pageCount(num)
        
        displayPage(start, total, entries, num)
        
window.close()